In [1]:
import torch
from torch import nn
tensor = torch.rand(4)
print(tensor)

t = torch.flatten(tensor)
loss = nn.BCELoss()
# res = loss(t, torch.tensor([1]).float())

res = loss(t, torch.full(tuple(t.shape), fill_value=1).float())
print(torch.mean(-torch.log(tensor))) 
print(type(res))

tensor([0.6712, 0.2613, 0.5673, 0.9451])
tensor(0.5910)
<class 'torch.Tensor'>


In [2]:
import os
import sys
from pathlib import Path

script_dir = os.getcwd()
DATA_FOLDER = os.path.join(script_dir, 'training', 'data')
current = script_dir
while 'src' not in os.listdir(current):
    current = Path(current).parent

sys.path.append(str(current))
sys.path.append(os.path.join(str(current), 'FaceSpoofing'))


In [3]:
import shutil
import src.pytorch_modular.directories_and_files as dirf


RAW_DATA = os.path.join(DATA_FOLDER, 'raw')
# if os.path.isdir(RAW_DATA):
#     shutil.rmtree(RAW_DATA)

# dirf.unzip_data_file(os.path.join(DATA_FOLDER, 'raw.zip'))
# dirf.squeeze_directory(RAW_DATA)
# # remove any non directories in RAW_DATA folder
# # rename
# os.rename(os.path.join(RAW_DATA, 'ClientRaw'), os.path.join(RAW_DATA, 'real'))
# os.rename(os.path.join(RAW_DATA, 'ImposterRaw'), os.path.join(RAW_DATA, 'fake'))

# initial_raw_data = os.listdir(RAW_DATA)
# for f in initial_raw_data:
#     pf = os.path.join(RAW_DATA, f)
#     if not os.path.isdir(pf):
#         os.remove(pf)

# raw1 = os.path.join(DATA_FOLDER, 'raw', 'real')
# raw0 = os.path.join(DATA_FOLDER, 'raw', 'fake')


# initial_dirs1 = os.listdir(raw1)
# initial_dirs0 = os.listdir(raw0)


In [4]:
# for raw1_dir in initial_dirs1:
#     path_dir = os.path.join(raw1, raw1_dir)
#     if os.path.isdir(path_dir) and not str(raw1_dir).endswith('.db'):
#         dirf.copy_directories(path_dir, raw1, copy=False, filter_directories=lambda x: not x.endswith('.db'))   
#     else:
#         os.remove(path_dir)
#     # remove the directory at the end
#     shutil.rmtree(path_dir)
        
# for raw0_dir in initial_dirs0:
#     path_dir = os.path.join(raw0, raw0_dir)

#     if os.path.isdir(path_dir) and not str(raw0_dir).endswith('.db'):
#         dirf.copy_directories(path_dir, raw0,copy=False, filter_directories=lambda x: not x.endswith('.db'))
#     else:
#         os.remove(path_dir)
#     shutil.rmtree(path_dir)



In [5]:
# let's put everything together
TRAIN_DIR = os.path.join(DATA_FOLDER, 'train')
TEST_DIR = os.path.join(DATA_FOLDER, 'test')

# if os.path.isdir(TRAIN_DIR):
#     shutil.rmtree(TRAIN_DIR)
#     os.makedirs(TRAIN_DIR)

# if os.path.isdir(TEST_DIR):
#     shutil.rmtree(TEST_DIR)
#     os.makedirs(TEST_DIR)

In [6]:
# pass a portion of the dataset to the training dir

# dirf.dataset_portion(RAW_DATA, TRAIN_DIR, portion=0.9, copy=False)
# dirf.copy_directories(RAW_DATA, TEST_DIR, copy=False)


In [7]:
from FaceSpoofing.PixBiSupervision.training import preprocessing as pr, losses as ls
from  FaceSpoofing.PixBiSupervision.model import PiBiNet as binet,  DenseNetFE as dense
import importlib

importlib.reload(pr)
importlib.reload(ls)
importlib.reload(binet)
importlib.reload(dense)

# pr.prepare_data(TRAIN_DIR)
# pr.prepare_data(TEST_DIR)
# train_dl, test_dl = pr.prepare_dataloaders(TRAIN_DIR, TEST_DIR)

<module 'FaceSpoofing.PixBiSupervision.model.DenseNetFE' from '/home/ayhem18/DEV/My_Kaggle_Repo/FaceSpoofing/PixBiSupervision/model/DenseNetFE.py'>

In [8]:
# prepare the data loaders
from src.pytorch_modular.data_loaders import create_dataloaders
train_dl, test_dl, classes_info = create_dataloaders(train_dir=TRAIN_DIR, 
                                       test_dir=TEST_DIR,
                                       train_transform=binet.DenseNetFeatureExtractor.default_transform,
                                       batch_size=256)

In [9]:
# let's define the model
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR
model = binet.PiBiNet()
loss = ls.pixelBinaryLoss()
optimizer = AdamW(params=model.parameters(), lr=10**-4, weight_decay=10**-5)
lr_scheduler = LinearLR(optimizer=optimizer, start_factor=1, end_factor=0.5, total_iters=100)


def compute_loss(y_pred: torch.Tensor, y: torch.Tensor, alpha: float=0.5) -> torch.Tensor:
    bfm, logits = y_pred
    feature_map_loss = loss(bfm, y)
    binary_loss = nn.BCEWithLogitsLoss(logits, y)
    return alpha * feature_map_loss + binary_loss * (1 - alpha)

In [10]:
from torchmetrics import Accuracy
accuracy_metric = Accuracy(task='binary')
metrics = {'accuracy': accuracy_metric}

train_configuration = {'optimizer': optimizer,
                        'scheduler': lr_scheduler,
                        'min_val_loss': 10 ** -4,
                        'max_epochs': 5,
                        'report_epoch': 2,
                        'compute_loss': compute_loss,
                        'metrics': metrics,
                        'no_improve_stop': 10
                        }

In [11]:
import src.pytorch_modular.image_classification.engine_classification as cls
from  FaceSpoofing.PixBiSupervision.model import PiBiNet as binet,  DenseNetFE as dense
importlib.reload(cls)
importlib.reload(binet)
HOME = os.getcwd()

results = cls.train_model(model, 
                          train_dl, 
                          test_dl, 
                          train_configuration,          
                          log_dir=os.path.join(HOME, 'runs'),         
                          save_path=os.path.join(HOME, 'saved_models'))   

[INFO] Created SummaryWriter, saving to: /home/ayhem18/DEV/My_Kaggle_Repo/FaceSpoofing/PixBiSupervision/runs/experience_11...


  0%|          | 0/5 [01:08<?, ?it/s]
ERROR:tornado.general:SEND Error: Host unreachable
